In [1]:
import numpy as np

In [3]:
x = np.random.rand(10,1,28,28) # 높이 28, 너비 28, 채널 1개인 데이터가 10개
x.shape

(10, 1, 28, 28)

In [4]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

In [6]:
x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1, 5, 5, stride = 1, pad = 0)
col1

array([[0.94580872, 0.40563236, 0.53617088, 0.85868551, 0.23417332,
        0.80095522, 0.00752548, 0.87176876, 0.47180869, 0.90295746,
        0.60444863, 0.13758251, 0.70330708, 0.74421326, 0.5205181 ,
        0.62458113, 0.98590875, 0.57090816, 0.74695399, 0.3691273 ,
        0.76651323, 0.0144723 , 0.14661127, 0.38500543, 0.44475743,
        0.78401297, 0.21374712, 0.03001618, 0.44098598, 0.43591847,
        0.47183102, 0.01315931, 0.77721437, 0.96250243, 0.45457453,
        0.63963323, 0.75195067, 0.02459286, 0.3907011 , 0.93981126,
        0.01275936, 0.76872516, 0.97341694, 0.92669526, 0.59121568,
        0.20100345, 0.42436707, 0.46193179, 0.52267754, 0.80890273,
        0.09385824, 0.0268577 , 0.8062059 , 0.30491097, 0.90035043,
        0.42291898, 0.81853161, 0.52878266, 0.09653009, 0.81876863,
        0.64262884, 0.19259222, 0.85820361, 0.90798167, 0.12096991,
        0.08945618, 0.54226951, 0.94760024, 0.02822387, 0.62749362,
        0.36672685, 0.58792905, 0.98029266, 0.74

In [19]:
class Convolution :
    def __init__(self, W, b, stride = 1, pad =0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x) :
        # 필터 ( FN, C, FH, FW)
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1+ (H + 2*self.pad - FH) / self.stride)
        out_w = int(1+ (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self, stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # 필터 전개(ex, 7*7, c=3 필터 : 7*7*3 = 147) -> (147,1)로 전개
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2) # out의 인덱스 순서를 변경
        
        return out

In [17]:
class Pooling : 
    def __init__(self, pool_h, pool_w, stride =1, pad = 0):
        self.pool_h = poll_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1+ (H + 2*self.pad - FH) / self.stride)
        out_w = int(1+ (W + 2*self.pad - FW) / self.stride)
        
        # 전개(1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)
        
        # 최댓값(2)
        out = np.max(col, axis =1 )
        
        # 성형(3)
        out = out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)
        
        return out
    

In [18]:
class SimpleConvNet :
    def __init__(self, input_dim = (1,28,28),
                conv_param = {'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1}
                hidden_size=100, output_size=10, weight_init_std=0.01) :
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size + 2*filter_pad - filter_size) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        self.layers = OrderDict()
        self.layers['Conv1'] = Convolution(self.params['W1'],self.params['b1'], 
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h = 2, pool_w =2, stride =2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        
        self.last_layer = SoftmaxWithLoss()

SyntaxError: invalid syntax (<ipython-input-18-8d55f45c5731>, line 4)